In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import time

import numpy as np
import dask
from abcpy.core import *
from abcpy.distributions import *
from abcpy.examples.ma2 import MA2, autocov, distance
from distributed import Client
from dask.dot import dot_graph
from functools import partial

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline


In [ ]:
# Setup distributed
# client = Client()
# dask.set_options(get=client.get)

In [ ]:
n = 1000
t1 = 0.6
t2 = 0.2

# Set up observed data y
latents = np.random.randn(n+2)
y = MA2(n, 1, t1, t2, latents=latents)

# Plot
plt.figure(figsize=(11, 6));
plt.plot(np.arange(0,n),y[0,:]);
plt.scatter(np.arange(-2,n), latents);

In [ ]:
# Set up the simulator
simulator = partial(MA2, n)

# Set up autocovariance summaries
ac1 = partial(autocov, 1)
ac2 = partial(autocov, 2)

# Specify the graphical model
t1 = Prior('t1', 'uniform', 0, 1)
t2 = Prior('t2', 'uniform', 0, 1)
Y = Simulator('MA2', simulator, t1, t2, observed=y)
S1 = Summary('S1', ac1, Y)
S2 = Summary('S2', ac2, Y)
d = Discrepancy('d', distance, S1, S2)

# Specify the number of simulations
N = 2000000

# Time and run the simulator in parallel
s = time.time()
dists = d.generate(N, batch_size=5000).compute()

print("Elapsed time %d sec" % (time.time() - s))

# Take the parameters
t1_sample = t1.generate(N).compute()
t2_sample = t2.generate(N).compute()

In [ ]:
# Set threshold and reject to get posteriors
eps = 2
accepts = dists < eps
t1_post = t1_sample[accepts]
t2_post = t2_sample[accepts]
print("Number of accepted samples %d" % sum(accepts))

In [ ]:
if len(t1_post) > 0:
    print("Posterior for t1")
    plt.hist(t1_post, bins=20)
else:
    print("No accepted samples")

In [ ]:
if len(t2_post) > 0:
    print("Posterior for t2")
    plt.hist(t2_post, bins=20)
else:
    print("No accepted samples")